# Initialization

Change to `valuenet` directory and add `src` path to `PYTHONPATH`.

In [ ]:
%cd /home/ec2-user/SageMaker/valuenetforked

In [ ]:
import sys
sys.path.insert(0, '/home/ec2-user/SageMaker/valuenetforked/src')
sys.path

Read environment

In [ ]:
NER_API_SECRET=%env NER_API_SECRET
API_KEY=%env API_KEY
DB_USER=%env DB_USER
DB_PW=%env DB_PW
DB_HOST=%env DB_HOST
DB_PORT=%env DB_PORT
DB_SCHEMA="public"

# Train Model

Here also, we follow the [user manual from Valuenet](https://github.com/brunnurs/valuenet).

In [ ]:
# Empty gpu use
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi
# kill -9 PID

## Training data

The data that will be used for training is located in the files
- valuenet/data/**dataset**/train.json
- valuenet/data/**dataset**/dev.json

with **dataset** being either hack_zurich or spider.

### Train with Spider dataset

<span style="color:red">**TODO: test it**</span> need to get the sqlite db

Add the spider sqlite database in # TODO. Then set dataset to 'spider' (as the cell below).

In [ ]:
data_set = 'spider'

### Train with Custom Data only

<span style="color:red">Checked but **TODO: need PR to be merged**</span>

Create your custom data and preprocess it following the steps of [preprocess_custom_data-01.ipynb](https://github.com/hack-with-admin-ch/aws-sagemaker-notebook-valuenet/blob/main/preprocess_custom_data-01.ipynb), then set dataset to 'hack_zurich' (as the cell below).

In [ ]:
data_set = 'hack_zurich'

### Train with Spider Dataset and the Custom Data

<span style="color:red">**TODO: test it**</span> there might be issues because it won't know which db to use

In this case, merge your custom data and spider data together in one json fole for train and one for dev:
- valuenet/data/hack_zurich/train.json
- valuenet/data/hack_zurich/dev.json

In [ ]:
data_set = 'hack_zurich'

## Hyperparameters

Here, we set a few parameters but do not hesitate to look at [training parameters](https://github.com/brunnurs/valuenet/blob/hackzurich/src/config.py) if you want to specify other settings.

In [ ]:
# data_set = 'hack_zurich', (or 'spider') already specified above
exp_name = "train-01" # Name of the training (saved in wandb and folder name in experiments)
batch_size = 2 # make it smaller if you encounter memory limitations 
num_epochs = 2

## Training procedure

**Results and final model**:
After training, you will be able to find the ground truth data, output of the model, evaluation results and intermediary data in the folder `valuenet/experiments/{exp_name}_date_time`.
If the accuracy was better than the last best accuracy, the model is also saved in the folder as `"best_model.pt"`.

**Note**: There will be errors of the type below. It is expected and normal.

    Exception
        Traceback (most recent call last):
            File "/home/ec2-user/SageMaker/valuenetforked/src/intermediate_representation/sem2sql/sem2SQL.py", line 691, in transform_semQL_to_sql
                result = transform(sem_ql_prediction[i], schemas[sem_ql_prediction[i]['db_id']])
            File "/home/ec2-user/SageMaker/valuenetforked/src/intermediate_representation/sem2sql/sem2SQL.py", line 246, in transform
        assert isinstance(c_instance, Root1)
    AssertionError

### Train from scratch
From scratch (with exception of the transformer encoder, which is already pre-trained).

In [ ]:
%run src/main.py --exp_name {exp_name} --data_set {data_set} --cuda --batch_size {batch_size} --num_epochs {num_epochs} --loss_epoch_threshold 70 --sketch_loss_weight 1.0 --beam_size 1 --seed 90

### Fine-tune a pretrained model

This will only train on the custom data starting from the weights of the fully pretrained model with spider dataset.

First download the best model so far: `best_model.pt` that was trained on the spider dataset.

In [ ]:
!wget https://github.com/brunnurs/valuenet/releases/download/trained_model/best_model.pt
!mv best_model.pt pretrained_models/best_model.pt

You probably want to decrease the learning weights. (Note: The weights are not frozen in the current implementation. You have to choose the one you want to freeze and do it yourself if you want to).

In [ ]:
# As an indication, these are the default learning weights divided by 10 (this was not optimised, feel free to modify !)
lr_base = 1e-3/10
lr_connection = 1e-4/10
lr_transformer = 2e-5/10

Now, you can continue the training from the weights of the `best_model.pt`.

In [ ]:
%run src/main.py --exp_name {exp_name} --data_set {data_set} --fine_tuning --cuda --batch_size {batch_size} --num_epochs {num_epochs} --loss_epoch_threshold 70 --sketch_loss_weight 1.0 --beam_size 1 --seed 90 --lr_base {lr_base} --lr_connection {lr_connection} --lr_transformer {lr_transformer}